# Defining Libor Deposits, FRAs and Libor Futures

In [2]:
# This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
# without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  
# See the license for more details.

Simple introduction to the inputs required to define a Libor deposit and a FRA

In [62]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinMath import ONE_MILLION
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinCalendar import FinCalendarTypes
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.products.libor.FinLiborFRA import FinLiborFRA
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborFuture import FinLiborFuture

Before proceeding I am going to create a simple discount curve assuming flat zero rates. I will use this later.

In [51]:
from financepy.market.curves.FinFlatCurve import FinFlatCurve
discountCurve = FinFlatCurve(settlementDate, 0.04, 1)

## Libor Deposits


A deposit is used to borrow cash interbank for a period specified at a rate agreed today. It settles in spot days.

In [115]:
valuationDate = FinDate(1, 3, 2020)
spotDays = 0
settlementDate = valuationDate.addWorkDays(spotDays)

Interest is charged using a day count convention to measure the period

This is a twelve month deposit at 4%

In [116]:
notional = ONE_MILLION
depositRate = 0.04
dayCountType = FinDayCountTypes.ACT_360
maturityDate = settlementDate.addTenor("12M")
calendarType = FinCalendarTypes.TARGET

In [117]:
depo = FinLiborDeposit(settlementDate, maturityDate, depositRate, dayCountType, notional, calendarType)

In [118]:
print(depo)

SETTLEMENT DATE: SUN 1 MAR 2020
MATURITY DATE: MON 1 MAR 2021
NOTIONAL: 1000000
DEPOSIT RATE: 0.04
DAY COUNT TYPE: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.TARGET
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING



In [119]:
depo.value(valuationDate, discountCurve)

1000560.1892867951

It should be exactly 1M. However the discount curve is not using the same conventions as a Libor deposit which is why it is not.

## Floating Rate Agreements (FRAs)

A floating rate agreement allows the buyer to lock in a rate of interest for borrowing over a future period start on a future start date and ending on the future maturity date.

In [120]:
fraRate = 0.04
dayCountType = FinDayCountTypes.ACT_360
fraStartDate = settlementDate.addMonths(1)
fraMaturityDate = settlementDate.addMonths(4)
payFixed = True
notional = 100000
calendarType = FinCalendarTypes.TARGET

In [121]:
fra = FinLiborFRA(fraStartDate, fraMaturityDate, fraRate, dayCountType, notional, payFixed, calendarType)

In [122]:
print(fra)

START ACCD DATE: WED 1 APR 2020
MATURITY DATE: WED 1 JUL 2020
FRA RATE: 0.04
NOTIONAL: 100000
PAY FIXED RATE: True
DAY COUNT TYPE: FinDayCountTypes.ACT_360
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING
CALENDAR: FinCalendarTypes.TARGET



In [123]:
fra.value(valuationDate, discountCurve)

28.758257924311945

This is small given the notional of $1M. It should be exactly zero. However the discount curve is not using the same conventions as a Libor FRA which is why it is not.

## Interest Rate Futures

Interest rate futures can be set up easily. Just set the number of the future.

In [126]:
fut = FinLiborFuture(valuationDate, 1)

In [125]:
print(fut)

LAST TRADING DATE: MON 16 MAR 2020
DELIVERY DATE: WED 18 MAR 2020
END INTEREST PERIOD: WED 17 JUN 2020
ACCRUAL TYPE: FinDayCountTypes.ACT_360
CONTRACT SIZE: 1000000



This is the fifth futures contract.

In [127]:
fut = FinLiborFuture(valuationDate, 5)

In [128]:
print(fut)

LAST TRADING DATE: MON 15 MAR 2021
DELIVERY DATE: WED 17 MAR 2021
END INTEREST PERIOD: WED 16 JUN 2021
ACCRUAL TYPE: FinDayCountTypes.ACT_360
CONTRACT SIZE: 1000000



For curve building we need to convert these to FRAs. We also need to provide a FRA price and convexity adjustment.

In [133]:
fra = fut.toFRA(97.323,-0.006)

In [134]:
print(fra)

START ACCD DATE: WED 17 MAR 2021
MATURITY DATE: WED 16 JUN 2021
FRA RATE: 0.026710000000000067
NOTIONAL: 1000000
PAY FIXED RATE: False
DAY COUNT TYPE: FinDayCountTypes.ACT_360
BUS DAY ADJUST TYPE: FinBusDayAdjustTypes.MODIFIED_FOLLOWING
CALENDAR: FinCalendarTypes.WEEKEND

